In [1]:
import os
import sys
import numpy as np
from torch.utils.data import Dataset
from dvis import dvis

sys.path.append('utils')
sys.path.append('fsn')

import pc_util

from model_util_fsn import FSNDatasetConfig, rotate_aligned_boxes

DC = FSNDatasetConfig()
MAX_NUM_OBJ = 64
MEAN_COLOR_RGB = np.array([109.8, 97.2, 83.8])
from pathlib import Path
from dyme import Dyme


In [2]:
data_path = '/home/norman/datasets/scannet_4d'
all_scan_names = list(set([x for x in os.listdir(data_path) if x.startswith("scene")]))
DC = FSNDatasetConfig()

## load GT

In [3]:
scan_name = 'scene0000_00'
frame_idx = 0

def load_gt(scan_name, frame_idx, data_path=data_path, load_pc=True):
    dy = Dyme(Path(data_path,scan_name))
    if load_pc:
        pc_wa = dy.load(f"pc_wa/{frame_idx}.npz")
    else:
        pc_wa = None
    frame_anno3d = dy.load(f"annotation3d/{frame_idx}.yaml")

    gt_instances = []
    for inst_anno in frame_anno3d['instances'].values():
        if inst_anno["nyu40_id"] not in DC.nyu40ids:
            continue
        inst_anno.update({'sn18_id': DC.nyu40id2class[inst_anno["nyu40_id"]]})
        gt_instances.append(inst_anno) 
    return pc_wa, gt_instances

pc_wa, gt_instances = load_gt(scan_name, frame_idx, load_pc=True)


## load MaskRCNN

In [4]:
import pickle
#generate with predictions2bboxes.ipynb
eval_maskrcnn_fn =  '/home/norman/detectron2/scannet/fsn_eval.pkl'
eval_maskrcnn = pickle.load(open(eval_maskrcnn_fn, 'rb'))


In [45]:
sys.path.append('models/')
from ap_helper import APCalculator, parse_predictions, parse_groundtruths
from eval_det import get_iou




In [42]:
#pred_all: map of {img_id: [(classname, bbox, score)]}
#gt_all: map of {img_id: [(classname, bbox)]}

In [60]:
### compute 3d AP on all MASK RCNN predictions
ap_calculator_list = [APCalculator(iou_thresh, DC.class2type) for iou_thresh in [0.001,0.01,0.05,0.1]]
gt_ref = 'aabb_wa'
for counter, ((scan_name, frame_idx), mask_insts) in enumerate(eval_maskrcnn.items()):
    if counter > 300:
        break
    for ap_calculator in ap_calculator_list:
        pred_map_cls = [(mask_inst['category_id']-1, mask_inst['aabb_wa'], mask_inst['score']) for mask_inst in mask_insts]
        _, gt_instances = load_gt(scan_name, frame_idx, data_path=data_path, load_pc=False)
        gt_map_cls = [(gt_instance['sn18_id'], gt_instance[gt_ref]) for gt_instance in gt_instances]
        #print([x[0] for x in gt_map_cls])
        ap_calculator.step([pred_map_cls], [gt_map_cls])

for ap_calculator in ap_calculator_list:
    ret_dict = ap_calculator.compute_metrics(get_iou)
    print(ret_dict)

3 0.8846153846153846
2 0.0
7 0.0
8 0.0
0 nan
17 0.0
10 nan
4 0.10526315789473684
11 0
{'cabinet Average Precision': nan, 'chair Average Precision': 0.0, 'sofa Average Precision': 0.8846153846153846, 'table Average Precision': 0.10526315789473684, 'bookshelf Average Precision': 0.0, 'picture Average Precision': 0.0, 'desk Average Precision': nan, 'curtain Average Precision': 0, 'otherfurniture Average Precision': 0.0, 'mAP': nan, 'cabinet Recall': nan, 'chair Recall': 0.0, 'sofa Recall': 0.8846153846153846, 'table Recall': 0.10526315789473684, 'bookshelf Recall': 0.0, 'picture Recall': 0.0, 'desk Recall': nan, 'curtain Recall': 0, 'otherfurniture Recall': 0.0, 'AR': nan}
3 0.8846153846153846
2 0.0
7 0.0
8 0.0
0 nan
17 0.0
10 nan
4 0.10526315789473684
11 0
{'cabinet Average Precision': nan, 'chair Average Precision': 0.0, 'sofa Average Precision': 0.8846153846153846, 'table Average Precision': 0.10526315789473684, 'bookshelf Average Precision': 0.0, 'picture Average Precision': 0.0, 'des

In [61]:
### show some samples
dvis(load_gt(scan_name, frame_idx, data_path=data_path, load_pc=True)[0][:,:6], vs=0.01)
for pred_map in pred_map_cls:
    dvis(pred_map[1], c=pred_map[0], name=f"pred/{DC.class2type[pred_map[0]]}")
for gt_map in gt_map_cls:
    dvis(gt_map[1], c=gt_map[0],name=f"gt/{DC.class2type[gt_map[0]]}", vs=2)

Sending xyzrgb with shape (70731, 6)
Sending bbox with shape (6,)
Sending bbox with shape (6,)


In [53]:
### compute 3d AP on all MASK RCNN predictions
gt_ref = 'aabb_wa'
ap_calculator_list_det = [APCalculator(iou_thresh, DC.class2type) for iou_thresh in [0.001,0.01,0.05,0.1]]
for counter, ((scan_name, frame_idx), mask_insts) in enumerate(eval_maskrcnn.items()):
    for ap_calculator in ap_calculator_list_det:
        pred_map_cls = [(1, mask_inst['aabb_wa'], mask_inst['score']) for mask_inst in mask_insts]
        _, gt_instances = load_gt(scan_name, frame_idx, data_path=data_path, load_pc=False)
        gt_map_cls = [(1, gt_instance[gt_ref]) for gt_instance in gt_instances]
        #print([x[0] for x in gt_map_cls])
        ap_calculator.step([pred_map_cls], [gt_map_cls])


for ap_calculator in ap_calculator_list_det:
    ret_dict = ap_calculator.compute_metrics(get_iou)
    print(ret_dict)

1 0.15193577873442127
{'bed Average Precision': 0.15193577873442127, 'mAP': 0.15193577873442127, 'bed Recall': 0.2222222222222222, 'AR': 0.2222222222222222}
1 0.15193577873442127
{'bed Average Precision': 0.15193577873442127, 'mAP': 0.15193577873442127, 'bed Recall': 0.2222222222222222, 'AR': 0.2222222222222222}
1 0.14773997453861706
{'bed Average Precision': 0.14773997453861706, 'mAP': 0.14773997453861706, 'bed Recall': 0.21367521367521367, 'AR': 0.21367521367521367}
1 0.14773997453861706
{'bed Average Precision': 0.14773997453861706, 'mAP': 0.14773997453861706, 'bed Recall': 0.21367521367521367, 'AR': 0.21367521367521367}


In [29]:
## display faabb_wa
dvis(pc_wa[:,:6], vs=0.01, name=f'{frame_idx}/pc_wa')
for gt_inst in gt_instances:
    dvis(gt_inst['faabb_wa'], c=gt_inst['inst_id'], name=f"{frame_idx}/faabb_wa_{gt_inst['inst_id']}")


Sending xyzrgb with shape (69061, 6)
Sending bbox with shape (6,)
Sending bbox with shape (6,)
Sending bbox with shape (6,)
Sending bbox with shape (6,)
Sending bbox with shape (6,)


In [21]:
## display aabb_wa
dvis(pc_wa[:,:6], vs=0.01, name=f'{frame_idx}/pc_wa')
for gt_inst in gt_instances:
    dvis(gt_inst['aabb_wa'], c=gt_inst['inst_id'], name=f"{frame_idx}/aabb_wa_{gt_inst['inst_id']}")

Sending xyzrgb with shape (69061, 6)
Sending bbox with shape (6,)
Sending bbox with shape (6,)
Sending bbox with shape (6,)
Sending bbox with shape (6,)
Sending bbox with shape (6,)


## Load VoteNet

In [81]:
from dutils import compute_AABB

In [74]:
eval_votenet =  Dyme('/home/norman/votenet/eval_fsn2/').load('serialize_preds.yaml')

In [75]:
# changes in next
eval_vn = dict()
for eval_vote in eval_votenet:
    eval_vn[(eval_vote['scan_name'], eval_vote['frame_idx'])] = eval_vote['instances']

In [76]:
eval_vote['instances'][0].keys()

dict_keys(['cls_id', 'faabb', 'obj_prob'])

In [84]:
ap_calculator_list = [APCalculator(iou_thresh, DC.class2type) for iou_thresh in [0.001,0.01,0.05,0.1]]
gt_ref = 'faabb_wa'
for counter, ((scan_name, frame_idx), mask_insts) in enumerate(eval_vn.items()):
    if counter > 100:
        break
    for ap_calculator in ap_calculator_list:
        pred_map_cls = []
        for mask_inst in mask_insts:
            faabb_org = np.copy(mask_inst['faabb'])
            faabb = np.copy(faabb_org[:,[0,2,1]])
            faabb[:,2] *= -1
            aabb = compute_AABB(faabb)
        pred_map_cls.append((mask_inst['cls_id'], aabb, mask_inst['obj_prob']))
        _, gt_instances = load_gt(scan_name, frame_idx, data_path=data_path, load_pc=False)
        gt_map_cls = [(gt_instance['sn18_id'], gt_instance[gt_ref]) for gt_instance in gt_instances]
        #print([x[0] for x in gt_map_cls])
        ap_calculator.step([pred_map_cls], [gt_map_cls])

for ap_calculator in ap_calculator_list:
    ret_dict = ap_calculator.compute_metrics(get_iou)
    print(ret_dict)

2 0.0269892937412119
4 0.02150537634408602
1 nan
5 0.021739130434782608
3 0.14615384615384616
0 0.05753968253968254
14 nan
17 0.0
7 nan
15 0.0
6 0.0
8 0.0
12 0
9 0
{'cabinet Average Precision': 0.05753968253968254, 'bed Average Precision': nan, 'chair Average Precision': 0.0269892937412119, 'sofa Average Precision': 0.14615384615384616, 'table Average Precision': 0.02150537634408602, 'door Average Precision': 0.021739130434782608, 'window Average Precision': 0.0, 'bookshelf Average Precision': nan, 'picture Average Precision': 0.0, 'counter Average Precision': 0, 'refridgerator Average Precision': 0, 'toilet Average Precision': nan, 'sink Average Precision': 0.0, 'otherfurniture Average Precision': 0.0, 'mAP': nan, 'cabinet Recall': 0.08928571428571429, 'bed Recall': nan, 'chair Recall': 0.09316770186335403, 'sofa Recall': 0.15384615384615385, 'table Recall': 0.03225806451612903, 'door Recall': 0.043478260869565216, 'window Recall': 0.0, 'bookshelf Recall': nan, 'picture Recall': 0.0, 

In [79]:
mask_insts[0]['faabb']

array([[ 4.14664947, -0.9620785 ,  1.54509652],
       [ 4.14664947, -0.9620785 ,  0.24192464],
       [ 1.92554416, -0.9620785 ,  0.24192464],
       [ 1.92554416, -0.9620785 ,  1.54509652],
       [ 4.14664947, -2.9970885 ,  1.54509652],
       [ 4.14664947, -2.9970885 ,  0.24192464],
       [ 1.92554416, -2.9970885 ,  0.24192464],
       [ 1.92554416, -2.9970885 ,  1.54509652]])

In [86]:
### find matching scan_name, frame_idx
common_ids = set(eval_vn.keys()).intersection(eval_maskrcnn)




In [91]:
mask_ap_calculator_list = [APCalculator(iou_thresh, DC.class2type) for iou_thresh in [0.001,0.01,0.05,0.1]]
vn_ap_calculator_list = [APCalculator(iou_thresh, DC.class2type) for iou_thresh in [0.001,0.01,0.05,0.1]]
gt_ref = 'aabb_wa'#'faabb_wa'


for counter, (scan_name, frame_idx) in enumerate(common_ids):
    # calc gt
    _, gt_instances = load_gt(scan_name, frame_idx, data_path=data_path, load_pc=False)
    gt_map_cls = [(gt_instance['sn18_id'], gt_instance[gt_ref]) for gt_instance in gt_instances]
    # calc maskrcnn
    mask_insts = eval_maskrcnn[(scan_name, frame_idx)]
    mask_pred_map_cls = [(1, mask_inst['aabb_wa'], mask_inst['score']) for mask_inst in mask_insts]

    for ap_calculator in mask_ap_calculator_list:
        ap_calculator.step([mask_pred_map_cls], [gt_map_cls])

    # calc votenet
    vt_insts = eval_vn[(scan_name, frame_idx)]
    vn_pred_map_cls = []
    for vt_inst in vt_insts:
        faabb_org = np.copy(vt_inst['faabb'])
        faabb = np.copy(faabb_org[:,[0,2,1]])
        faabb[:,2] *= -1
        aabb = compute_AABB(faabb)
    vn_pred_map_cls.append((vt_inst['cls_id'], aabb, vt_inst['obj_prob']))

    for ap_calculator in vn_ap_calculator_list:
        ap_calculator.step([vn_pred_map_cls], [gt_map_cls])


In [92]:
for ap_calculator in mask_ap_calculator_list:
    ret_dict = ap_calculator.compute_metrics(get_iou)
    print(ret_dict)

1 nan
3 0
4 0
{'bed Average Precision': nan, 'sofa Average Precision': 0, 'table Average Precision': 0, 'mAP': nan, 'bed Recall': nan, 'sofa Recall': 0, 'table Recall': 0, 'AR': nan}
1 nan
3 0
4 0
{'bed Average Precision': nan, 'sofa Average Precision': 0, 'table Average Precision': 0, 'mAP': nan, 'bed Recall': nan, 'sofa Recall': 0, 'table Recall': 0, 'AR': nan}
1 nan
3 0
4 0
{'bed Average Precision': nan, 'sofa Average Precision': 0, 'table Average Precision': 0, 'mAP': nan, 'bed Recall': nan, 'sofa Recall': 0, 'table Recall': 0, 'AR': nan}
1 nan
3 0
4 0
{'bed Average Precision': nan, 'sofa Average Precision': 0, 'table Average Precision': 0, 'mAP': nan, 'bed Recall': nan, 'sofa Recall': 0, 'table Recall': 0, 'AR': nan}


In [93]:
for ap_calculator in vn_ap_calculator_list:
    ret_dict = ap_calculator.compute_metrics(get_iou)
    print(ret_dict)

2 nan
3 0.21052631578947367
5 nan
4 0.14814814814814814
1 nan
{'bed Average Precision': nan, 'chair Average Precision': nan, 'sofa Average Precision': 0.21052631578947367, 'table Average Precision': 0.14814814814814814, 'door Average Precision': nan, 'mAP': nan, 'bed Recall': nan, 'chair Recall': nan, 'sofa Recall': 0.21052631578947367, 'table Recall': 0.2222222222222222, 'door Recall': nan, 'AR': nan}
2 nan
3 0.21052631578947367
5 nan
4 0.14814814814814814
1 nan
{'bed Average Precision': nan, 'chair Average Precision': nan, 'sofa Average Precision': 0.21052631578947367, 'table Average Precision': 0.14814814814814814, 'door Average Precision': nan, 'mAP': nan, 'bed Recall': nan, 'chair Recall': nan, 'sofa Recall': 0.21052631578947367, 'table Recall': 0.2222222222222222, 'door Recall': nan, 'AR': nan}
2 nan
3 0.21052631578947367
5 nan
4 0.037037037037037035
1 nan
{'bed Average Precision': nan, 'chair Average Precision': nan, 'sofa Average Precision': 0.21052631578947367, 'table Average 